In [7]:
import torch

In [2]:
from torch.nn import Conv1d

In [3]:
kernel_size = 3

dilatedConv = Conv1d(1, 1, kernel_size=kernel_size, dilation=2)

In [4]:
params = list(dilatedConv.parameters())
weight, bias = params[0].data, params[1].data

In [8]:
example_tensor = torch.tensor([[[1.,2.,3.,4.,5.,6.,7.,8.,9.,10.]]])

In [6]:
dilatedConv(example_tensor)

tensor([[[3.3973, 3.8965, 4.3957, 4.8949, 5.3942, 5.8934]]],
       grad_fn=<ConvolutionBackward0>)

In [9]:
num_examples, kernal_shape =  example_tensor.shape[-1], weight.shape[-1]
dilation = 2

activations = torch.tensor([])
for i in range(num_examples -kernal_shape-1):
    data = example_tensor[:,:,i:i+kernal_shape*dilation:dilation]
    activation = data.squeeze(0) @ weight.squeeze_(0).T + bias
    print(activation)
    activations = torch.cat((activations, activation))

print(activations)

tensor([3.3973])
tensor([3.8965])
tensor([4.3957])
tensor([4.8949])
tensor([5.3942])
tensor([5.8934])
tensor([3.3973, 3.8965, 4.3957, 4.8949, 5.3942, 5.8934])


In [10]:
def dialtedConv(inputs, weight, bias):
    num_examples, kernal_shape =  inputs.shape[-1], weight.shape[-1]
    
    activations = torch.tensor([])
    for i in range(num_examples -kernal_shape-1):
        data = inputs[:,:,i:i+kernal_shape*dilation:dilation]
        activation = data.squeeze(0) @ weight.squeeze_(0).T + bias
        activations = torch.cat((activations, activation))
        
    return activations.view(1,1,-1)

dialtedConv(example_tensor, weight, bias)   

tensor([[[3.3973, 3.8965, 4.3957, 4.8949, 5.3942, 5.8934]]])

In [11]:
import torch.nn as nn

In [12]:
layers_dilation = [2**i for i in range(10)]
layers_dilation

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]

In [36]:
stacked_layers = [Conv1d(1, 1, padding=0, kernel_size=kernel_size, dilation=dilation) for dilation in layers_dilation]
stacked_layers

[Conv1d(1, 1, kernel_size=(3,), stride=(1,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(2,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(4,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(8,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(16,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(32,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(64,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(128,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(256,)),
 Conv1d(1, 1, kernel_size=(3,), stride=(1,), dilation=(512,))]

In [49]:
example_tensor_stacked = torch.arange(1, 512*4, dtype=torch.float).view(1,1,-1)
example_tensor_stacked

tensor([[[1.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 2.0450e+03,
          2.0460e+03, 2.0470e+03]]])

In [50]:
activations = example_tensor_stacked
for layer in stacked_layers:
    print(activations.shape)
    activations = layer(activations)

torch.Size([1, 1, 2047])
torch.Size([1, 1, 2045])
torch.Size([1, 1, 2041])
torch.Size([1, 1, 2033])
torch.Size([1, 1, 2017])
torch.Size([1, 1, 1985])
torch.Size([1, 1, 1921])
torch.Size([1, 1, 1793])
torch.Size([1, 1, 1537])
torch.Size([1, 1, 1025])


In [4]:
import torch.nn as nn

class DialatedConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, bias=True):
        super(DialatedConv1d, self).__init__(in_channels, out_channels, kernel_size,
                                             dilation=dilation, bias=bias)
    def forward(self, inputs):
        return super(DialatedConv1d, self).forward(inputs)

In [9]:
DialatedConv1d(1, 1, 3, 2, True)(example_tensor)

tensor([[[2.2176, 2.7410, 3.2645, 3.7879, 4.3113, 4.8348]]],
       grad_fn=<ConvolutionBackward0>)